In [1]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
#
# This source code is licensed under the MIT license found in the
# LICENSE file in the root directory of this source tree.

import time
import pickle

import numpy as np
import sophus as sp
import cv2
import torch

import polymetis

from teleop_input_device import OculusTeleop
from teleop_robot import TeleopRobotCartesianControl
from realsense_d435 import RealSense

In [2]:
# Metainfo
NUC_IP = "100.96.135.68"

UPDATE_HZ = 30
HOME_POSE = [0.0748543,-0.13675,-0.401604,-2.10836,0.287622,1.76446,-1.34985]

NUM_EPISODES = 10
MAX_EPISODE_DURATION = 10 #seconds

In [3]:
# Initialize interfaces
print("Connecting to devices...")
robot = TeleopRobotCartesianControl(ip_address=NUC_IP, use_gripper=False, home_pose=HOME_POSE)
print("Connected to robot.")
teleop = OculusTeleop(query_hz=UPDATE_HZ)
print("Connected to teleop device.")
camera = RealSense()
print("Connected to camera.")

Connecting to devices...
Connected to robot.
Connected to teleop device.
Connecting to RealSense cameras (1 found) ...
Connected to camera 1 (745412070129).
Connected to camera.


In [4]:
# Helper functions
def pose_elementwise_diff(pose1, pose2):
    return sp.SE3(
        (pose2.so3() * pose1.so3().inverse()).matrix().T,
        pose2.translation() - pose1.translation(),
    )

def pose_elementwise_apply(delta_pose, pose):
    return sp.SE3(
        (pose.so3() * delta_pose.so3()).matrix(),
        delta_pose.translation() + pose.translation(),
    )

def compute_desired_pose(vr_pose_curr, vr_pose_ref, arm_pose_ref):
    """vr_pose_curr - vr_pose_ref = arm_pose_desired - arm_pose_ref"""
    vr_pose_diff = pose_elementwise_diff(vr_pose_ref, vr_pose_curr)
    arm_pose_desired = pose_elementwise_apply(vr_pose_diff, arm_pose_ref)
    return arm_pose_desired

In [ ]:
# Initialize variables
vr_pose_ref = sp.SE3()
arm_pose_ref = sp.SE3()
init_episode = True

t0 = time.time()
t_target = t0
t_delta = 1.0 / UPDATE_HZ

episode_count = 0
episode_length = 0
max_episode_length = UPDATE_HZ * MAX_EPISODE_DURATION

demo_data = []

# Start teleop loop
print("======================== TELEOP START =========================")
while episode_count < NUM_EPISODES:
    # Obtain info from teleop device
    is_active, vr_pose_curr, grasp_state = teleop.get_state()

    # Update arm
    if is_active:
        # Initialize episode
        if init_episode:
            print(f"Starting episode {episode_count + 1}.")

            # Initialize reference pose
            vr_pose_ref = vr_pose_curr
            arm_pose_ref = robot.get_ee_pose()
            init_episode = False

            # Initialize episode data
            demo_data.append([])

        # Query & save state data
        img = camera.get_images()[0]
        joint_angles = robot.get_joint_pos()
        if episode_length <= max_episode_length:
            demo_data[episode_count].append((img, joint_angles))
        if episode_length == max_episode_length:
            print(f"\tWarning: Max episode length of {max_episode_length} reached.")

        # Update
        arm_pose_desired = compute_desired_pose(vr_pose_curr, vr_pose_ref, arm_pose_ref)
        robot.update_ee_pose(arm_pose_desired)
        robot.update_grasp_state(grasp_state)

        episode_length += 1

    else:
        if not init_episode:
            print(f"\tEnd of episode {episode_count + 1}. Resetting...")
            init_episode = True
            robot.reset()
            print("Done.")

            # Update stats
            episode_count += 1
            episode_length = 0


    # Spin once
    t_target += t_delta
    t_remaining = t_target - time.time()
    time.sleep(max(t_remaining, 0.0))
    
print("======================== TELEOP END =========================")

======================== TELEOP START =========================
Starting episode 1.
	End of episode 1. Resetting...
Done.
Starting episode 2.
Interrupt detected. Reinstantiating control policy...
	End of episode 2. Resetting...
Done.
Starting episode 3.
Interrupt detected. Reinstantiating control policy...
	End of episode 3. Resetting...
Done.
Starting episode 4.
Interrupt detected. Reinstantiating control policy...
Interrupt detected. Reinstantiating control policy...
	End of episode 4. Resetting...
Done.
Starting episode 5.
Interrupt detected. Reinstantiating control policy...
	End of episode 5. Resetting...
Done.


In [8]:
# Downsample imgs
demo_data_processed = [
    [(cv2.resize(state[0], (160, 120)), state[1].numpy()) for state in episode]
    for episode in demo_data
]

In [9]:
# Save data
item_num = 15
with open(f"demos_item{item_num}.pkl", 'wb') as f:
    pickle.dump(demo_data_processed, f)